In [ ]:
!pip install rouge_score

In [1]:
import os
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

import torch
DEVICE = torch.device("mps") if torch.backends.mps.is_available() else 'cpu'
DEVICE

/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='mps')

In [2]:
from datasets import load_dataset, load_metric

# Load the dataset
dataset = load_dataset('scientific_papers', 'arxiv')

# Get the training, validation and test datasets
train_dataset = dataset['train']
val_dataset = dataset['validation']
test_dataset = dataset['test']

/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/datasets/load.py:1429: FutureWarning: The repository for scientific_papers contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/scientific_papers
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [3]:
from transformers import AutoTokenizer
from transformers import BartTokenizer

model_path = "facebook/bart-large-cnn"
#tokenizer = AutoTokenizer.from_pretrained(model_path)
tokenizer = BartTokenizer.from_pretrained(model_path)

max_input_length = 1024
max_output_length = 128
batch_size = 2

def process_data_to_model_inputs(batch):    
    # tokenize the inputs and labels
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=max_input_length)
    outputs = tokenizer(batch["abstract"], padding="max_length", truncation=True, max_length=max_output_length)
    
    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask
    batch["labels"] = outputs.input_ids

    # ignore the PAD token
    batch["labels"] = [[-100 if token == tokenizer.pad_token_id else token for token in labels] for labels in
                       batch["labels"]]
    
    return batch

train_dataset = train_dataset.select(range(250))
val_dataset = val_dataset.select(range(25))

train_dataset = train_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

val_dataset = val_dataset.map(
    process_data_to_model_inputs,
    batched=True,
    batch_size=batch_size,
    remove_columns=["article", "abstract", "section_names"],
)

train_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)
val_dataset.set_format(
    type="torch", columns=["input_ids", "attention_mask", "labels"]
)

Map: 100%|██████████| 25/25 [00:01<00:00, 19.48 examples/s]


In [ ]:
train_dataset[2]["article"]

In [6]:
import numpy as np

task_dataset = train_dataset
for i in range(len(task_dataset)):
    ls = task_dataset[i]["input_ids"].numpy()
    indices = np.where(ls == 2)
    if len(indices) > 1:
        print(i)
        print(indices)

In [7]:
from transformers import AutoModelForSeq2SeqLM
from transformers import BartForConditionalGeneration

model = AutoModelForSeq2SeqLM.from_pretrained(model_path, gradient_checkpointing=True, use_cache=False)
#model = BartForConditionalGeneration.from_pretrained(model_path, gradient_checkpointing=True, use_cache=False)

# set hyperparameters
model.config.num_beams = 2
model.config.max_length = 128
model.config.min_length = 80
model.config.length_penalty = 2.0
model.config.early_stopping = True
model.config.no_repeat_ngram_size = 3

In [8]:
rouge = load_metric("rouge")

def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    labels_ids[labels_ids==-100] = tokenizer.pad_token_id
    label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    rouge_output = rouge.compute(
        predictions=pred_str, references=label_str, rouge_types=["rouge2"] 
    )["rouge2"].mid 

    return {
        "rouge2_precision": round(rouge_output.precision, 4),
        "rouge2_recall": round(rouge_output.recall, 4),
        "rouge2_fmeasure": round(rouge_output.fmeasure, 4)
    }

/var/folders/yy/p134q1115s35c2fqxthfm4_c0000gn/T/ipykernel_52876/1956387722.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge = load_metric("rouge")
/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/datasets/load.py:752: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [9]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    predict_with_generate=True,
    evaluation_strategy="steps",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    #fp16=True,
    use_mps_device=True,
    output_dir="./",
    logging_steps=5,
    eval_steps=10,
    save_steps=10,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
)

trainer = Seq2SeqTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [10]:
trainer.train()

/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 250
  Num Epochs = 1
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 4
  Total optimization steps = 31
  Number of trainable parameters = 406290432
 16%|█▌        | 5/31 [00:29<02:20,  5.40s/it]

{'loss': 3.5405, 'learning_rate': 4.1935483870967746e-05, 'epoch': 0.16}


 32%|███▏      | 10/31 [00:54<01:49,  5.21s/it]***** Running Evaluation *****
  Num examples = 25
  Batch size = 2


{'loss': 2.9054, 'learning_rate': 3.387096774193548e-05, 'epoch': 0.32}


/Users/i0545652/article_summary/.venv/lib/python3.8/site-packages/transformers/generation_utils.py:2797: UserWarning: The operator 'aten::remainder.Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  next_tokens = next_tokens % vocab_size
                                               
 32%|███▏      | 10/31 [07:32<01:49,  5.21s/it]Saving model checkpoint to ./checkpoint-10
Configuration saved in ./checkpoint-10/config.json


...

                                               
 97%|█████████▋| 30/31 [23:26<00:11, 11.01s/it]Saving model checkpoint to ./checkpoint-30
Configuration saved in ./checkpoint-30/config.json


{'eval_loss': 2.640104055404663, 'eval_rouge2_precision': 0.1284, 'eval_rouge2_recall': 0.1252, 'eval_rouge2_fmeasure': 0.1247, 'eval_runtime': 369.0262, 'eval_samples_per_second': 0.068, 'eval_steps_per_second': 0.035, 'epoch': 0.96}


Model weights saved in ./checkpoint-30/pytorch_model.bin
tokenizer config file saved in ./checkpoint-30/tokenizer_config.json
Special tokens file saved in ./checkpoint-30/special_tokens_map.json
Deleting older checkpoint [checkpoint-10] due to args.save_total_limit
100%|██████████| 31/31 [23:58<00:00, 128.01s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 31/31 [23:58<00:00, 46.42s/it] 

{'train_runtime': 1438.9551, 'train_samples_per_second': 0.174, 'train_steps_per_second': 0.022, 'train_loss': 2.9066912820262294, 'epoch': 0.99}


TrainOutput(global_step=31, training_loss=2.9066912820262294, metrics={'train_runtime': 1438.9551, 'train_samples_per_second': 0.174, 'train_steps_per_second': 0.022, 'train_loss': 2.9066912820262294, 'epoch': 0.99})